---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 2 - Introduction to NLTK

In part 1 of this assignment you will use nltk to explore the Herman Melville novel Moby Dick. Then in part 2 you will create a spelling recommender function that uses nltk to find words similar to the misspelling. 

## Part 1 - Analyzing Moby Dick

In [ ]:
import nltk
import pandas as pd
import numpy as np

nltk.download('punkt')
# If you would like to work with the raw text you can use 'moby_raw'
with open('moby.txt', 'r') as f:
    moby_raw = f.read()
    
# If you would like to work with the novel in nltk.Text format you can use 'text1'
moby_tokens = nltk.word_tokenize(moby_raw)
text1 = nltk.Text(moby_tokens)

In [ ]:
text1

### Example 1

How many tokens (words and punctuation symbols) are in text1?

*This function should return an integer.*

In [ ]:
def example_one():
    
    return len(nltk.word_tokenize(moby_raw)) # or alternatively len(text1)

example_one()

### Example 2

How many unique tokens (unique words and punctuation) does text1 have?

*This function should return an integer.*

In [ ]:
def example_two():
    
    return len(set(nltk.word_tokenize(moby_raw))) # or alternatively len(set(text1))

example_two()

### Example 3

After lemmatizing the verbs, how many unique tokens does text1 have?

*This function should return an integer.*

In [ ]:
from nltk.stem import WordNetLemmatizer

def example_three():

    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w,'v') for w in text1]

    return len(set(lemmatized))

example_three()

### Question 1

What is the lexical diversity of the given text input? (i.e. ratio of unique tokens to the total number of tokens)

*This function should return a float.*

In [ ]:
def answer_one():
    
    
    return len(set(nltk.word_tokenize(moby_raw)))/len(nltk.word_tokenize(moby_raw)) # Your answer here

answer_one()

### Question 2

What percentage of tokens is 'whale'or 'Whale'?

*This function should return a float.*

In [ ]:
def answer_two():
    
    
    from nltk import FreqDist
    
    dist = FreqDist(text1)
    
    whale = dist['whale']
    Whale = dist['Whale']
    
    return ((whale + Whale) / len(nltk.word_tokenize(moby_raw))) * 100 # Your answer here

answer_two()

### Question 3

What are the 20 most frequently occurring (unique) tokens in the text? What is their frequency?

*This function should return a list of 20 tuples where each tuple is of the form `(token, frequency)`. The list should be sorted in descending order of frequency.*

In [ ]:
def answer_three():
    
    from nltk import FreqDist

    dist = FreqDist(text1)

    token_sorted = sorted(dist.items(), key = lambda x:x[1], reverse = True)
    
    return token_sorted[:20] # Your answer here

answer_three()

### Question 4

What tokens have a length of greater than 5 and frequency of more than 150?

*This function should return an alphabetically sorted list of the tokens that match the above constraints. To sort your list, use `sorted()`*

In [ ]:
def answer_four():
    from nltk import FreqDist

    dist = FreqDist(text1)
    
    tokens = dist.keys()
    freqwords = [w for w in tokens if len(w) > 5 and dist[w] > 150]
    
    return sorted(freqwords) # Your answer here

answer_four()

### Question 5

Find the longest word in text1 and that word's length.

*This function should return a tuple `(longest_word, length)`.*

In [ ]:
def answer_five():
    
    from nltk import FreqDist

    dist = FreqDist(text1)
    tokens = dist.keys()
    length = 0
    word = ''
    
    for w in tokens:
        if len(w) > length:
            length = len(w)
            word = w            
    
    return (word, length) # Your answer here

answer_five()

### Question 6

What unique words have a frequency of more than 2000? What is their frequency?

"Hint:  you may want to use `isalpha()` to check if the token is a word and not punctuation."

*This function should return a list of tuples of the form `(frequency, word)` sorted in descending order of frequency.*

In [ ]:
def answer_six():
    
    #2000개 이상 빈도 (단어여야하고 구두점 아니여야함.)
    from nltk import FreqDist

    dist = FreqDist(text1)
    tokens = dist.keys()
    return_lst = []
    
    for w in tokens:
        if dist[w] > 2000:
            if w.isalpha():
                return_lst.append((dist[w], w))
    
    return sorted(return_lst, key = lambda x:x[0], reverse=True) # Your answer here

answer_six()

### Question 7

What is the average number of tokens per sentence?

*This function should return a float.*

In [ ]:
def answer_seven():
    
    sentences = nltk.sent_tokenize(moby_raw)
    token_sum = 0
    
    for sen in sentences:
        token_sum += len(nltk.word_tokenize(sen))
        
    
    return token_sum / len(sentences) # Your answer here

answer_seven()

### Question 8

What are the 5 most frequent parts of speech in this text? What is their frequency?

*This function should return a list of tuples of the form `(part_of_speech, frequency)` sorted in descending order of frequency.*

In [ ]:
def answer_eight():
    from itertools import groupby

    nltk.download('averaged_perceptron_tagger')
    pos = nltk.word_tokenize(moby_raw)
    pos_list = nltk.pos_tag(pos)

    transformed_list = []
    d = {}
    pos_name = []
    final_list = []

    for tup in pos_list:
        transformed_list.append((tup[1],1))

    for a in transformed_list:
        if not a[0] in pos_name:
            pos_name.append(a[0])
            d[a[0]] = 0
        d[a[0]] += a[1]

    for pos in pos_name:
        final_list.append((pos, d[pos]))
    
    return sorted(final_list, key = lambda x: x[1], reverse=True)# Your answer here

answer_eight()

## Part 2 - Spelling Recommender

For this part of the assignment you will create three different spelling recommenders, that each take a list of misspelled words and recommends a correctly spelled word for every word in the list.

For every misspelled word, the recommender should find find the word in `correct_spellings` that has the shortest distance*, and starts with the same letter as the misspelled word, and return that word as a recommendation.

*Each of the three different recommenders will use a different distance measure (outlined below).

Each of the recommenders should provide recommendations for the three default words provided: `['cormulent', 'incendenece', 'validrate']`.

In [ ]:
from nltk.corpus import words
nltk.download('words')

correct_spellings = words.words()

In [ ]:
correct_spellings[:10]

### Question 9

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the trigrams of the two words.**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [ ]:
def answer_nine(entries=['cormulent', 'incendenece', 'validrate']):
    
    c_list = [c for c in correct_spellings if c[0] == 'c']
    i_list = [i for i in correct_spellings if i[0] == 'i']
    v_list = [v for v in correct_spellings if v[0] == 'v']
    
    #distance and word
    c_list2 = [(nltk.jaccard_distance(set(nltk.ngrams(entries[0], n=3)),
                                      set(nltk.ngrams(a, n=3))), a) for a in c_list]

    i_list2 = [(nltk.jaccard_distance(set(nltk.ngrams(entries[1], n=3)),
                                      set(nltk.ngrams(a, n=3))), a) for a in i_list]
    
    v_list2 = [(nltk.jaccard_distance(set(nltk.ngrams(entries[2], n=3)),
                                      set(nltk.ngrams(a, n=3))), a) for a in v_list]
    
    answer = [sorted(c_list2)[0][1], sorted(i_list2)[0][1], sorted(v_list2)[0][1]]
    return answer # Your answer here
    
answer_nine()

### Question 10

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the 4-grams of the two words.**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [ ]:
def answer_ten(entries=['cormulent', 'incendenece', 'validrate']):
    
    c_list = [c for c in correct_spellings if c[0] == 'c']
    i_list = [i for i in correct_spellings if i[0] == 'i']
    v_list = [v for v in correct_spellings if v[0] == 'v']
    
    #distance and word
    c_list2 = [(nltk.jaccard_distance(set(nltk.ngrams(entries[0], n=4)),
                                      set(nltk.ngrams(a, n=4))), a) for a in c_list]

    i_list2 = [(nltk.jaccard_distance(set(nltk.ngrams(entries[1], n=4)),
                                      set(nltk.ngrams(a, n=4))), a) for a in i_list]
    
    v_list2 = [(nltk.jaccard_distance(set(nltk.ngrams(entries[2], n=4)),
                                      set(nltk.ngrams(a, n=4))), a) for a in v_list]
    
    answer = [sorted(c_list2)[0][1], sorted(i_list2)[0][1], sorted(v_list2)[0][1]]
    
    return answer # Your answer here
    
answer_ten()

### Question 11

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Edit distance on the two words with transpositions.](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance)**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [ ]:
def answer_eleven(entries=['cormulent', 'incendenece', 'validrate']):
    
    c_list = [c for c in correct_spellings if c[0] == 'c']
    i_list = [i for i in correct_spellings if i[0] == 'i']
    v_list = [v for v in correct_spellings if v[0] == 'v']
    
    #distance and word
    c_list2 = [(nltk.edit_distance(entries[0],a), a) for a in c_list]

    i_list2 = [(nltk.edit_distance(entries[1],a), a) for a in i_list]
    
    v_list2 = [(nltk.edit_distance(entries[2],a), a) for a in v_list]
    
    answer = [sorted(c_list2)[0][1], sorted(i_list2)[0][1], sorted(v_list2)[0][1]]
    
    return answer # Your answer here 
    
answer_eleven()